In [13]:
import time
import pygame as py
import copy
import math

In [14]:
def Trophy():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [15]:
def End():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [16]:
def Violet():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [17]:
def Red():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [18]:
def Yellow():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [19]:
def Green():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [20]:
def Coin():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [21]:
def Orange():
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        


    return(running)

In [22]:
def Blue(width, height, coins, font, stick):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(width/8, height-height/25)
    jumping = False
    attack = False
    
    # Health
    blue_hp = 1
    player_hp = 5
    colours = {'blue':True, 'orange':True, 'green': True, 'yellow':True, 'red':True, 'violet': True}
    
    # Blue Dialogue
    whining = ['*The square continues to cry*', '*Blue sniffles*', '*Blue stops crying*']
    stages = -1
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        
    # Creating Background
        screen.fill("white")
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws Blue Square
        if blue_hp != 0:
            py.draw.rect(screen, "blue", (7*width/8, 11*height/12, height/12, height/12))
            py.draw.rect(screen, "black", (7*width/8, 11*height/12, height/12, height/12), 3)


        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d]:
            if blue_hp != 0 and player_pos.x < 3*width/4:
                player_pos.x += 300 * dt
            if blue_hp == 0 and player_pos.x < width-height/25:
                player_pos.x += 300 * dt
    
            
        """# Player Engages With Blue Square
        if player_pos.x > width and stages <= 2 and attack == False:
            hint = font.render('(do not hold) PRESS 1 [talk]', True, 'black')
            screen.blit(hint, (width/8, height/3))
            
            if keys[py.K_1]:
                stages += 1
                time.sleep(0.3)
        
        if 0 <= stages <= 5:
            advice = font.render(dialogue[stages], True, 'black')
            screen.blit(advice, (width/8, height/2))"""
            
        
        
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        
        if player_pos.x >= width-height/20:
            running = Orange(width, height, coins, font, stick, player_hp)

    
    return(running)

In [23]:
def Gray(width, height, coins, font):
    
    # Collection of Variables
    dt = 0
    running = True
    player_pos = py.Vector2(7*width/8, height-height/25)
    jumping = False
    
    # Gray Dialogue
    dialogue = ['Hello Adventurer', 'You must unite the colours', 'Travel east-ward and you will encounter them', 'Be warned, each is more dangerous', 'May this stick help you', 'Good luck Adventurer']
    option = -1
    stick = False
    
    # Gameplay
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                return False
        
        # Creating Background
        screen.fill("white")
        
        # Draws Player
        py.draw.circle(screen, "indigo", player_pos, height/23, 0)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        # Draws Gray Square
        py.draw.rect(screen, "grey", (width/5, 11*height/12, height/12, height/12))
        py.draw.rect(screen, "black", (width/5, 11*height/12, height/12, height/12), 3)
                

        keys = py.key.get_pressed()
        
        # Player Jumps
        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            # Fixes a Teleportation Glitch
            dt = clock.tick(60) / 1000
            dt = 0
            
        # Circle is Being Pulled by Gravity
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
        
        # Circle has Landed
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
    
            
        # Player Engages With Gray Square
        if player_pos.x < width/3 and option <= 5:
            hint = font.render('(do not hold) PRESS 1 [talk]', True, 'black')
            screen.blit(hint, (width/8, height/3))
            
            if keys[py.K_1]:
                option += 1
                time.sleep(0.3)
        
        if 0 <= option <= 5:
            advice = font.render(dialogue[option], True, 'black')
            screen.blit(advice, (width/8, height/2))
            
        if option >= 4:
            stick = True
        
            
        # Updates Screen
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        
        if player_pos.x >= width-height/20:
            running = Blue(width, height, coins, font, stick)

    
    return(running)

In [24]:
# Basic Setup
py.init()
py.font.init()

# FPS set-up
clock = py.time.Clock()

# Screen Parameters
width = 1000
height = 700

# Creating Font
font = py.font.SysFont('timesnewroman.ttf', int(width/30))

# Sets Up Window
screen = py.display.set_mode((width, height))
game1 = True


def Start():
    
    dt = 0
    running = True
    player_pos = py.Vector2(width/2, 17*height/18)

    # Displaying Text
    text = font.render('Turn Back', True, 'black')
    
    # Text
    py.display.set_caption('Colours')
    
    reveal = False
    jumping = False
    pickup = False
    coins = 0
    
    while running:
        # Closes Program
        for event in py.event.get():
            if event.type == py.QUIT:
                running = False
        
        # Creating Visuals
        screen.fill("white")
        py.draw.circle(screen, "indigo", player_pos, height/23, 0)
        py.draw.circle(screen, "black", player_pos, height/23, 3)
        
        
        if player_pos.x >= width-height/20:
            reveal = True
        
    
        if reveal == True and pickup == False:
            py.draw.circle(screen, "yellow", [width/10, 34*height/35], height/35, 0)
            py.draw.circle(screen, "black", [width/10, 34*height/35], height/35, 3)
            
                
        if reveal == True and width/10-height/18 <= player_pos.x <= width/10+height/18 and jumping == False:
            reveal = False
            coins = 1
            pickup = True
                

        keys = py.key.get_pressed()
        # Player Jumps

        if keys[py.K_w] and player_pos.y == 24*height/25:

            jumping = True
            
            dt = clock.tick(60) / 1000
            dt = 0
            
        if jumping and player_pos.y <= 24*height/25:
            player_pos.y -= (velocity*dt) + (-200*dt**2)
            velocity = velocity + (-400*dt)
            
        else:
            jumping = False
            player_pos.y = 24*height/25
            velocity = 3*height/5
            

        # Player Moves Horizontally
        if keys[py.K_a] and player_pos.x > height/25:
            player_pos.x -= 300 * dt
            
        if keys[py.K_d] and player_pos.x < width-height/25:
            player_pos.x += 300 * dt
    
            
        py.display.flip()
        
        # limits FPS to 60
        # dt is delta time in seconds since last frame, used for framerate
        dt = clock.tick(60) / 1000
        
        if player_pos.x <= height/20:
            running = Gray(width, height, coins, font)
        

if game1 == True:
    Start()
    game1 == False
    
py.quit()